In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from datetime import date
from tqdm import tqdm_notebook
from tqdm import tqdm
tqdm_notebook().pandas()

import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings('ignore')

A Jupyter Widget

In [2]:
ddd = {}
ddd['products'] = [5411, 5499]
ddd['meal'] = [5812, 5814]
ddd['bank'] = [6011, 6010]
ddd['apteka'] = [5912]
ddd['hotel'] = [7011]
ddd['parking'] = [7523]
ddd['alhogol'] = [5921]
ddd['car'] = [5541, 5533]
ddd['univermag'] = [5311, 5999, 5331]
ddd['entertainment'] = [7832, 7932, 7933]
ddd['sport'] = [5941]
ddd['hair'] = [7230]
ddd['cosm'] = [5977]
ddd['clothes'] = [5699, 5651, 5661, 5641]
ddd['child_clo'] = [5641]
ddd['zoomag'] = [5995]
mcc_list = ['products', 'meal', 'bank', 'apteka', 'hotel', 'parking', 'alhogol', 'car', 'univermag',\
            'entertainment', 'sport', 'hair', 'cosm', 'clothes', 'child_clo', 'zoomag']

In [3]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')
samp = pd.read_csv('sample.csv')

In [4]:
train.rename(columns={'pos_adress_lat':'pos_address_lat', 'pos_adress_lon':'pos_address_lon'}, inplace=True)

In [5]:
data = pd.concat(
[train[['amount', 'city', 'atm_address_lat', 'atm_address_lon', 'terminal_id', 'pos_address_lat', 'pos_address_lon','mcc']],
test[['amount', 'city', 'atm_address_lat', 'atm_address_lon', 'terminal_id', 'pos_address_lat', 'pos_address_lon','mcc']]])

In [ ]:
data.fillna(-1, inplace=True)

In [ ]:
data['lat'] = data.progress_apply(lambda x: max(x['atm_address_lat'], x['pos_address_lat']), axis=1)
data['lon'] = data.progress_apply(lambda x: max(x['atm_address_lon'], x['pos_address_lon']), axis=1)

A Jupyter Widget

A Jupyter Widget

In [ ]:
data = data[data.lat > 0]

In [ ]:
data

In [ ]:
uniq_term = data.terminal_id.unique()

In [ ]:
std_lat = data.groupby('terminal_id').apply(lambda x: max(x['lat']) - min(x['lat'])).to_dict()
std_lon = data.groupby('terminal_id').apply(lambda x: max(x['lon']) - min(x['lon'])).to_dict()

data['mcc'] = data['mcc'].apply(lambda x: int(str(x).replace(',', '')))
mcc = data.groupby('terminal_id').mcc.apply(lambda x: max(x)).to_dict()

In [ ]:
df = pd.DataFrame({'terminal_id':uniq_term})

In [ ]:
df['std_lat'] = df.terminal_id.map(std_lat)
df['std_lon'] = df.terminal_id.map(std_lon)

In [ ]:
lat_mean = data.groupby('terminal_id').lat.mean().to_dict()
lon_mean = data.groupby('terminal_id').lon.mean().to_dict()

In [ ]:
df['lat'] = df.terminal_id.map(lat_mean)
df['lon'] = df.terminal_id.map(lon_mean)

In [ ]:
df = df[(df.std_lat < 0.008) & (df.std_lon < 0.008)]

In [ ]:
df['mcc'] = df.terminal_id.map(mcc)

In [ ]:
df

In [ ]:
mcc_list2 = ['products', 'meal', 'bank', 'apteka', 'univermag','clothes']

r002 = []
r002_pr = []
r002_meal = []
r002_bank = []
r002_apt = []
r002_univ = []
r002_clo = []

for line in tqdm_notebook(np.array(df)):
    lat = line[3]
    lon = line[4]
    
    df['y'] = np.sqrt((df.lat - lat)**2 + (df.lon - lon)**2)
    r002.append(df[df.y < 0.02].shape[0])
    r002_pr.append(df[(df.y < 0.02) & ((df.mcc == 5411) | (df.mcc == 5499)) ].shape[0])
    r002_meal.append(df[(df.y < 0.02) & ((df.mcc == 5821) | (df.mcc == 5814)) ].shape[0])
    r002_bank.append(df[(df.y < 0.02) & ((df.mcc == 6011) | (df.mcc == 6010)) ].shape[0])
    r002_apt.append(df[(df.y < 0.02) & (df.mcc == 5912)].shape[0])
    r002_univ.append(df[(df.y < 0.02) & ((df.mcc == 7832) | (df.mcc == 7932) | (df.mcc == 7933)) ].shape[0])
    r002_clo.append(df[(df.y < 0.02) & ((df.mcc == 5699) | (df.mcc == 5651) | (df.mcc == 5661) | (df.mcc == 5641)) ].shape[0])
    
    
df['r002'] = r002
df['r002_pr'] = r002_pr
df['r002_meal'] = r002_meal
df['r002_bank'] = r002_bank
df['r002_apt'] = r002_apt
df['r002_univ'] = r002_univ
df['r002_clo'] = r002_clo

In [ ]:
df.to_csv('popularity.csv', index=False)

In [4]:
hhh = pd.concat([train[['terminal_id', 'transaction_date', 'customer_id']], test[['terminal_id', 'transaction_date', 'customer_id']]])

In [5]:
uniq_term = hhh.terminal_id.unique()

In [6]:
hhh['transaction_date'] = hhh['transaction_date'].apply(lambda x: str(x).lower())
hhh.transaction_date = pd.to_datetime(hhh.transaction_date)
hhh['dow'] = hhh.transaction_date.dt.dayofweek
hhh['month'] = hhh.transaction_date.dt.month
hhh.transaction_date = hhh.transaction_date.dt.date
hhh['hol'] = hhh.dow.apply(lambda x: int(x>4))


In [7]:
hhh.head(3)

,terminal_id,transaction_date,customer_id,dow,month,hol
0,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,0dc0137d280a2a82d2dc89282450ff1b,5.0,7.0,1
1,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,0dc0137d280a2a82d2dc89282450ff1b,4.0,10.0,0
2,df06c1fcd3718a514535ae822785f716,2017-10-03,0dc0137d280a2a82d2dc89282450ff1b,1.0,10.0,0


In [8]:
hol_per = hhh.groupby('terminal_id').hol.mean().to_dict()

In [9]:
hol_per_no_sum = hhh[(hhh.month != 6) & (hhh.month != 7) & (hhh.month != 8)].groupby('terminal_id').hol.mean().to_dict()

In [ ]:
num = hhh.groupby('terminal_id').hol.count().to_dict()
num_hol = hhh.groupby('terminal_id').hol.sum().to_dict()

In [ ]:
uniq_cust = hhh.groupby('terminal_id').customer_id.nunique().to_dict()

In [ ]:
df2 = pd.DataFrame({'terminal_id':uniq_term})

In [124]:
# df2['hol_per'] = df2.terminal_id.map(hol_per)
# df2['hol_per_no_sum'] = df2.terminal_id.map(hol_per_no_sum)
# df2['num'] = df2.terminal_id.map(num)
# df2['num_hol'] = df2.terminal_id.map(num_hol)
# df2['uniq_cust'] = df2.terminal_id.map(uniq_cust)

In [126]:
# df2.to_csv('terminal_holidays.csv', index=False)

In [11]:
hol_per = hhh.groupby('terminal_id').hol.mean().to_dict()
num = hhh.groupby('terminal_id').hol.count().to_dict()
num_hol = hhh.groupby('terminal_id').hol.sum().to_dict()
uniq_cust = hhh.groupby('terminal_id').customer_id.nunique().to_dict()

In [12]:
hol_per_win = hhh[(hhh.month ==12) | (hhh.month ==1) | (hhh.month ==2)].groupby('terminal_id').hol.mean().to_dict()
num_win = hhh[(hhh.month ==12) | (hhh.month ==1) | (hhh.month ==2)].groupby('terminal_id').hol.count().to_dict()
num_hol_win = hhh[(hhh.month ==12) | (hhh.month ==1) | (hhh.month ==2)].groupby('terminal_id').hol.sum().to_dict()
uniq_cust_win = hhh[(hhh.month ==12) | (hhh.month ==1) | (hhh.month ==2)].groupby('terminal_id').customer_id.nunique().to_dict()

In [13]:
hol_per_spr = hhh[(hhh.month ==3) | (hhh.month ==4) | (hhh.month ==5)].groupby('terminal_id').hol.mean().to_dict()
num_spr = hhh[(hhh.month ==3) | (hhh.month ==4) | (hhh.month ==5)].groupby('terminal_id').hol.count().to_dict()
num_hol_spr = hhh[(hhh.month ==3) | (hhh.month ==4) | (hhh.month ==5)].groupby('terminal_id').hol.sum().to_dict()
uniq_cust_spr = hhh[(hhh.month ==3) | (hhh.month ==4) | (hhh.month ==5)].groupby('terminal_id').customer_id.nunique().to_dict()

In [14]:
hol_per_sum = hhh[(hhh.month ==6) | (hhh.month ==7) | (hhh.month ==8)].groupby('terminal_id').hol.mean().to_dict()
num_sum = hhh[(hhh.month ==6) | (hhh.month ==7) | (hhh.month ==8)].groupby('terminal_id').hol.count().to_dict()
num_hol_sum = hhh[(hhh.month ==6) | (hhh.month ==7) | (hhh.month ==8)].groupby('terminal_id').hol.sum().to_dict()
uniq_cust_sum = hhh[(hhh.month ==6) | (hhh.month ==7) | (hhh.month ==8)].groupby('terminal_id').customer_id.nunique().to_dict()

In [16]:
hol_per_aut = hhh[(hhh.month == 9) | (hhh.month ==10) | (hhh.month ==11)].groupby('terminal_id').hol.mean().to_dict()
num_aut = hhh[(hhh.month == 9) | (hhh.month ==10) | (hhh.month ==11)].groupby('terminal_id').hol.count().to_dict()
num_hol_aut = hhh[(hhh.month == 9) | (hhh.month ==10) | (hhh.month ==11)].groupby('terminal_id').hol.sum().to_dict()
uniq_cust_aut = hhh[(hhh.month == 9) | (hhh.month ==10) | (hhh.month ==11)].groupby('terminal_id').customer_id.nunique().to_dict()

In [17]:
df2 = pd.DataFrame({'terminal_id':uniq_term})

In [18]:
df2['hol_per'] = df2.terminal_id.map(hol_per)
df2['num'] = df2.terminal_id.map(num)
df2['num_hol'] = df2.terminal_id.map(num_hol)
df2['uniq_cust'] = df2.terminal_id.map(uniq_cust)
df2['hol_per_win'] = df2.terminal_id.map(hol_per_win)
df2['num_win'] = df2.terminal_id.map(num_win)
df2['num_hol_win'] = df2.terminal_id.map(num_hol_win)
df2['uniq_cust_win'] = df2.terminal_id.map(uniq_cust_win)
df2['hol_per_spr'] = df2.terminal_id.map(hol_per_spr)
df2['num_spr'] = df2.terminal_id.map(num_spr)
df2['num_hol_spr'] = df2.terminal_id.map(num_hol_spr)
df2['uniq_cust_spr'] = df2.terminal_id.map(uniq_cust_spr)
df2['hol_per_sum'] = df2.terminal_id.map(hol_per_sum)
df2['num_sum'] = df2.terminal_id.map(num_sum)
df2['num_hol_sum'] = df2.terminal_id.map(num_hol_sum)
df2['uniq_cust_sum'] = df2.terminal_id.map(uniq_cust_sum)
df2['hol_per_aut'] = df2.terminal_id.map(hol_per_aut)
df2['num_aut'] = df2.terminal_id.map(num_aut)
df2['num_hol_aut'] = df2.terminal_id.map(num_hol_aut)
df2['uniq_cust_aut'] = df2.terminal_id.map(uniq_cust_aut)

In [21]:
print(df2.shape)
df2.head()

(290283, 21)


,terminal_id,hol_per,num,num_hol,uniq_cust,hol_per_win,num_win,num_hol_win,uniq_cust_win,hol_per_spr,num_spr,num_hol_spr,uniq_cust_spr,hol_per_sum,num_sum,num_hol_sum,uniq_cust_sum,hol_per_aut,num_aut,num_hol_aut,uniq_cust_aut
0,11606fde0c814ce78e0d726e39a0a5ee,0.500000,4.0,2.0,4.0,NaN,NaN,NaN,NaN,0.500000,2.0,1.0,2.0,1.00,1.0,1.0,1.0,0.000000,1.0,0.0,1.0
1,e9647a5e1eacfb06713b6af755ccc595,0.476923,65.0,31.0,50.0,NaN,NaN,NaN,NaN,0.523810,21.0,11.0,17.0,0.32,25.0,8.0,21.0,0.631579,19.0,12.0,19.0
2,df06c1fcd3718a514535ae822785f716,0.000000,3.0,0.0,2.0,NaN,NaN,NaN,NaN,0.000000,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0.000000,2.0,0.0,2.0
3,6c5e5793ebc984fb72875feffff62854,0.517241,58.0,30.0,43.0,NaN,NaN,NaN,NaN,0.368421,19.0,7.0,14.0,0.68,25.0,17.0,23.0,0.428571,14.0,6.0,12.0
4,0576445d74e374c92c0902e612fca356,0.250000,44.0,11.0,37.0,NaN,NaN,NaN,NaN,0.428571,7.0,3.0,5.0,0.25,24.0,6.0,22.0,0.153846,13.0,2.0,12.0


In [22]:
df2.to_csv('terminal_holidays_2.csv', index=False)